## 1 AI-Based News Summarization

In [ ]:
# !pip uninstall newspaper3k lxml
# !pip install lxml_html_clean
# !pip install newspaper3k

In [ ]:
from newspaper import Article
from transformers import pipeline

url = 'https://www.thehindu.com/news/the-hindu-morning-digest-march-20-2025/article69351767.ece'
article = Article(url)
article.download()
article.parse()

# print(article.text)

summarizer = pipeline('summarization')
summary = summarizer(article.text, max_length = 100, min_length = 20, do_sample = False)

print(summary[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


 IT Ministry examining issue of Grok using Hindi slang, abuses; in touch with social media platform X over the recent incident of AI chatbot Grok . Enforcement Directorate (ED) registered a total of 193 cases against politicians over the last 10 years and secured conviction in two of those, the government has informed Parliament .


## 2 Pdf Summarizer

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2
from transformers import pipeline, BartTokenizer

# Load and extract text from pdf
def extract_text(pdf_path):
  text = ''
  with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    for page in reader.pages:
      text += page.extract_text()
    return text

pdf_text = extract_text('ML_coursera_sup_proj.pdf')

# Summarization
summarizer = pipeline('summarization')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

tokens = tokenizer.encode(pdf_text, return_tensors='pt')

max_length = 1024
if tokens.shape[1] > max_length:
  tokens = tokens[:, :max_length]
  pdf_text = tokenizer.decode(tokens[0], skip_special_tokens=True)

print(pdf_text)

summary = summarizer(pdf_text, max_length = 250, min_length = 50, do_sample = False)

print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Linear Regression Project Report 
Main Objective of the Analysis
The main objective of this analysis is to predict housing prices using linear regression models. The 
focus is on creating an interpretable model that explains key factors driving housing prices. This 
analysis aims to provide actionable insights and recommendations for stakeholders based on the 
predictive power of the models.
Dataset Description
The dataset used for this analysis is the Ames Housing Dataset , a publicly available dataset 
containing comprehensive details about residential homes in Ames, Iowa. The dataset includes:
•Target Variable: SalePrice, representing the sale price of the house.
•Predictor Variables : Various numeric and categorical features such as LotArea, 
YearBuilt, OverallQual, etc., capturing the characteristics of the house and its 
surroundings.
Dataset Overview:
•Number of Rows: 2,930
•Number of Features: 81
•Numeric Features: 38
•Categorical Features: 43
Missing Data Handling:
•Columns wi

## 3 Youtube Transcript Summarization

In [ ]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.9 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from urllib.parse import urlparse, parse_qs


# Extract Youtube video Id seperately

def extract_video_id(video_url):
  parsed_text = urlparse(video_url)  # split the url in sub urls
  # print(parsed_text.hostname, parsed_text)

  if parsed_text.hostname in ['www.youtube.com', 'youtube.com', 'm.youtube.com']:
    query = parse_qs(parsed_text.query)
    # print(query)
    if 'v' in query:
      return query['v'][0]
    elif 'youtu.be' in parsed_text.netloc:   # Handle Youtube shorts
      return parsed_text.path[1:]
  elif parsed_text.hostname == 'youtu.be':
    return parsed_text.path[1:]
  else:
    return None


# Get the transcript of the video and summarize the texts in it.

def summarization(video_url):
  video_id = extract_video_id(video_url)   # Extract video id from using function
  if video_id:
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    transcript_text = ' '.join([entry['text'] for entry in transcript])
    print(len(transcript_text))
    print(transcript_text)


    max_length = min(int(len(transcript_text) * 0.4), 1024)
    min_length = min(int(len(transcript_text)*0.2), 512)
    # max_length = 512
    # min_length = 100
    print( min_length, max_length)

    model_name = 'google/long-t5-tglobal-base'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    inputs = tokenizer.encode(transcript_text, return_tensors='pt')


    summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)
    summary = summarizer(transcript_text, max_length = max_length, min_length = min_length, do_sample= False)
    print(summary[0]['summary_text'])
    print(len(summary[0]['summary_text']))


  else:
    return "Invalid URL"


video_url = 'https://youtu.be/eCDTp5h59hg?si=fdG03gNGkkixzXVJ'
summarization(video_url)


RequestBlocked: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=eCDTp5h59hg! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-ipblocked-exception).
2. (NOT RECOMMENDED) If you authenticate your requests using cookies, you will be able to continue doing requests for a while. However, YouTube will eventually permanently ban the account that you have used to aut

hi everyone welcome back to our channel in this video we will discuss some great computer vision projects without further Ado let's start with number 20 building a robotic arm that can pick up and sort objects is no easy task but Kai and his friends took on the challenge of developing their own automated system their goal to detect colored balls and sort them accurately using a budget friendly robotic manipulator they use the open manipulator x a lowcost robotic arm with four joints giving it 4° of movement a standard USB camera served as the robot's eye identifying the balls based on their color to ensure precise movement they first calculated the arms position using forward and inverse kinematics to sort the balls they corrected the camera's Distortion and processed the image to detect colors and positions once everything synced the robot seamlessly scanned grabbed and sorted the balls number 19 imagine walking past a booth at a maker fair and a robot suddenly locks eyes with you tra

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

Device set to use cpu


[{'summary_text': "number 13 if you've ever wanted to build your own AI powered camera this project is a great place to start Eric designed a simple yet effective setup using an esp32 cam inside a cardboard box along with a push button to capture images and a screen to display them but how does it recognize objects here's how it works the ESP 32 cam captures an image and uploads it to a nodejs server which then calls the Google Vision API for image analysis once the object is identified the processed image is sent back to the ESP 32 cam providing detailed information about what it sees with this setup you can easily identify and analyze objects in real time number 12 meet auto bill a fast and effective setup to generate shopping bills thus reducing the checkout time this project might look like a product but it is completely DIY it identifies the objects using a camera placed above while the load cell at the bottom weighs the object the object along with its weight price and quantity i

In [ ]:
print(len(summary[0]['summary_text']))

1191
